### Exemples de query possible sur DuckDB pour de l'analyses

#### Pour acceder aux données de DuckDB en local depuis le volume montée :
kubectl apply -f jupyter-pod.yaml
kubectl port-forward pod/jupyter-duckdb 8888:8888 -n airflow
#### Pour choper le token :
kubectl logs pod/jupyter-duckdb -n airflow

In [14]:
import duckdb

# Connexion au fichier DuckDB monté dans le pod
conn = duckdb.connect("analytics1.duckdb")

In [15]:
# Liste des tables disponibles
conn.sql("SHOW TABLES").df()

,name
0,gold_notif_impact_per_day
1,gold_user_activity_per_day


1) Afficher les 10 premières lignes de gold_notif_impact_per_day

In [16]:
conn.sql("SELECT * FROM gold_user_activity_per_day LIMIT 10").df()

,user_id,date,total_nbr_connexion,total_spend_time,total_message_send_this_day,total_money_spend
0,001fd039-7272-4e18-a324-26122f82c8fc,2025-02-17,1.0,0 days 02:45:36.788000,0.0,0.0
1,001fd039-7272-4e18-a324-26122f82c8fc,2025-02-21,3.0,0 days 12:02:45.509000,0.0,0.0
2,001fd039-7272-4e18-a324-26122f82c8fc,2025-02-24,1.0,0 days 02:26:37.430000,0.0,0.0
3,001fd039-7272-4e18-a324-26122f82c8fc,2025-02-26,1.0,0 days 01:50:47.416000,0.0,0.0
4,001fd039-7272-4e18-a324-26122f82c8fc,2025-02-28,1.0,0 days 01:28:20.312000,0.0,0.0
5,001fd039-7272-4e18-a324-26122f82c8fc,2025-03-01,1.0,0 days 02:28:11.008000,0.0,0.0
6,001fd039-7272-4e18-a324-26122f82c8fc,2025-03-02,2.0,0 days 21:04:45.519000,0.0,0.0
7,001fd039-7272-4e18-a324-26122f82c8fc,2025-03-03,2.0,0 days 16:37:41.018000,0.0,0.0
8,001fd039-7272-4e18-a324-26122f82c8fc,2025-03-04,1.0,0 days 01:55:41.956000,0.0,0.0
9,001fd039-7272-4e18-a324-26122f82c8fc,2025-03-05,1.0,0 days 01:50:39.148000,0.0,0.0


2) Durée moyenne de session après notification

In [17]:
conn.sql("""
    SELECT 
        notif_date, 
        ROUND(AVG(EXTRACT(EPOCH FROM time_spend_after_success))/60, 2) AS avg_time_spent_minutes
    FROM gold_notif_impact_per_day
    GROUP BY notif_date
    ORDER BY notif_date DESC
""").df()

,notif_date,avg_time_spent_minutes


3) Top contenus de notifications les plus engageants

In [18]:
conn.sql("""
    SELECT 
        content_notif,
        COUNT(*) AS nb_notifs,
        ROUND(AVG(EXTRACT(EPOCH FROM time_spend_after_success))/60, 2) AS avg_time_minutes
    FROM gold_notif_impact_per_day
    GROUP BY content_notif
    ORDER BY avg_time_minutes DESC
    LIMIT 10
""").df()

,content_notif,nb_notifs,avg_time_minutes


4) Taux de succès de notifications (notifications vues)

In [19]:
conn.sql("""
    SELECT 
        notif_date,
        SUM(CASE WHEN is_success THEN 1 ELSE 0 END) AS seen,
        COUNT(*) AS total,
        ROUND(100.0 * SUM(CASE WHEN is_success THEN 1 ELSE 0 END) / COUNT(*), 2) AS success_rate
    FROM gold_notif_impact_per_day
    GROUP BY notif_date
    ORDER BY notif_date
""").df()

,notif_date,seen,total,success_rate


5) Sessions actives par jour et par utilisateur

In [20]:
conn.sql("""
    SELECT 
        user_id,
        session_date,
        COUNT(*) AS nb_sessions
    FROM gold_user_activity_per_day
    GROUP BY user_id, session_date
    ORDER BY session_date DESC, nb_sessions DESC
""").df()

BinderException: Binder Error: Referenced column "session_date" not found in FROM clause!
Candidate bindings: "total_spend_time", "user_id", "total_message_send_this_day", "total_money_spend"